# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)
@show(a)

10000000-element Vector{Float64}:
 0.24185609479515446
 0.8072406051162906
 0.6273478925522273
 0.3340845690013656
 0.1795955821136821
 0.12010620701853036
 0.9506400644225175
 0.7233956347214138
 0.4101284058263607
 0.8227873768717585
 ⋮
 0.46333662399487563
 0.8203745102176433
 0.5290651244870812
 0.9757869643146375
 0.4113130937174816
 0.05257561130727362
 0.7671977075250968
 0.14343750310476078
 0.7157700747738306

In [2]:
sum(a)

4.999036941056104e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [3]:
@time sum(a)

  0.005016 seconds (1 allocation: 16 bytes)


4.999036941056104e6

In [7]:
@time sum(a)

  0.009075 seconds (1 allocation: 16 bytes)


4.999036941056104e6

In [23]:
@time sum(a)

  0.004180 seconds (1 allocation: 16 bytes)


4.999036941056104e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [9]:
using Pkg
Pkg.add("BenchmarkTools")

   Resolving package versions...
  No Changes to `/usr/local/julia/Project.toml`
  No Changes to `/usr/local/julia/Manifest.toml`


In [10]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

Let's assume that the current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [12]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [13]:
c_sum(a)

4.999036941056545e6

In [14]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [15]:
c_sum(a) - sum(a)  

4.4051557779312134e-7

In [16]:
≈  # alias for the `isapprox` function

isapprox (generic function with 13 methods)

In [18]:
# ?isapprox
# use this instead
display("text/markdown", Base.Docs.doc(isapprox))



```
isapprox(x, y; atol::Real=0, rtol::Real=atol>0 ? 0 : √eps, nans::Bool=false[, norm::Function])
```

Inexact equality comparison. Two numbers compare equal if their relative distance *or* their absolute distance is within tolerance bounds: `isapprox` returns `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` (absolute tolerance) is zero and the default `rtol` (relative tolerance) depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significant digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

The `norm` keyword defaults to `abs` for numeric `(x,y)` and to `LinearAlgebra.norm` for arrays (where an alternative `norm` choice is sometimes useful). When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

!!! compat "Julia 1.6"
    Passing the `norm` keyword argument when comparing numeric (non-array) arguments requires Julia 1.6 or later.


# Examples

```jldoctest
julia> isapprox(0.1, 0.15; atol=0.05)
true

julia> isapprox(0.1, 0.15; rtol=0.34)
true

julia> isapprox(0.1, 0.15; rtol=0.33)
false

julia> 0.1 + 1e-10 ≈ 0.1
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0]) # using `norm`
true
```

```
isapprox(x; kwargs...) / ≈(x; kwargs...)
```

Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.

The keyword arguments supported here are the same as those in the 2-argument `isapprox`.

!!! compat "Julia 1.5"
    This method requires Julia 1.5 or later.



We can now benchmark the C code directly from Julia:

In [21]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 636 samples with 1 evaluation.
 Range (min … max):  6.954 ms …  11.591 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.584 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.848 ms ± 774.515 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▅██▅▃▂                                                    
  ▄████████▆▇█▅▆▅▆▇▅▆▇▅▆▅▆▅▆▅▅▄▃▃▃▄▄▂▄▃▃▃▃▃▃▂▃▄▃▂▃▃▃▂▃▃▃▁▃▂▃▃ ▄
  6.95 ms         Histogram: frequency by time        10.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [22]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 6.953621 msec


In [24]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any, Any} with 1 entry:
  "C" => 6.95362

In [25]:
using Plots
gr()

Plots.GRBackend()

In [26]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

# 2. C with -ffast-math

If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [28]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum_fastmath (generic function with 1 method)

In [30]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

BenchmarkTools.Trial: 947 samples with 1 evaluation.
 Range (min … max):  3.978 ms …   9.546 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.975 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.269 ms ± 963.602 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃▃▇▅██▅▆▅▆▃▂ ▃▂                                           
  ▃▄████████████████▇▇▆▆▆▇▄▅▅▄▇▄▄▄▃▅▄▃▄▄▃▃▁▄▂▃▃▃▃▃▃▂▃▃▃▂▃▁▁▂▃ ▄
  3.98 ms         Histogram: frequency by time         8.7 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [21]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

1.781208

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [35]:
using Pkg; Pkg.add("PyCall")
using PyCall

   Resolving package versions...
    Updating `/usr/local/julia/Project.toml`
  [438e738f] + PyCall v1.96.4
    Updating `/usr/local/julia/Manifest.toml`
  [438e738f] + PyCall v1.96.4


In [36]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [37]:
pysum(a)

4.9997343666773755e6

In [38]:
pysum(a) ≈ sum(a)

true

In [39]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  425.922 ms … 5.658 s  ┊ GC (min … max):  0.00% … 32.04%
 Time  (median):     438.387 ms            ┊ GC (median):     0.00%
 Time  (mean ± σ):      1.091 s ± 1.846 s  ┊ GC (mean ± σ):  20.78% ± 11.33%

  █                                                           
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  426 ms         Histogram: frequency by time         5.66 s <

 Memory estimate: 152.60 MiB, allocs estimate: 10000038.

In [28]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "C"               => 9.38288
  "Python built-in" => 875.812
  "C -ffast-math"   => 1.78121

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [40]:
using Pkg; Pkg.add("Conda")
using Conda

   Resolving package versions...
    Updating `/usr/local/julia/Project.toml`
  [8f4d0f93] + Conda v1.10.2
  No Changes to `/usr/local/julia/Manifest.toml`


In [41]:
Conda.add("numpy")

┌ Info: Downloading miniconda installer ...
└ @ Conda /home/juan/.julia/packages/Conda/zReqD/src/Conda.jl:289


PREFIX=/home/juan/.julia/conda/3/x86_64


┌ Info: Installing miniconda ...
└ @ Conda /home/juan/.julia/packages/Conda/zReqD/src/Conda.jl:300



Transaction

  Prefix: /home/juan/.julia/conda/3/x86_64/envs/_virtual_specs_checks

  All requested packages already installed

Dry run. Not executing the transaction.
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2024.8.30-hbcca054_0.conda
Extracting ld_impl_linux-64-2.40-hf3520f5_7.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.12-5_cp312.conda
Extracting tzdata-2024a-h8827d51_1.conda
Extracting libgomp-14.1.0-h77fa898_1.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-14.1.0-h77fa898_1.conda
Extracting libexpat-2.6.3-h5888daf_0.conda
Extracting libgcc-ng-14.1.0-h69a702a_1.conda
Extracting libstdcxx-14.1.0-hc0a3c3a_1.conda
Extracting openssl-3.3.2-hb9d3cd8_0.conda
Extracting bzip2-1.0.8-h4bc722e_7.conda
Extracting c-ares-1.32.3-h4bc722e_0.conda
Extracting keyutils-1.6.1-h166bdaf_0.tar.bz2
Extracting libev-4.33-hd590300_2.conda
Extracting libffi-3.4.2-h7f98852_5.tar.bz2
Extracting libico

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda /home/juan/.julia/packages/Conda/zReqD/src/Conda.jl:181


Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: /home/juan/.julia/conda/3/x86_64

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libblas-3.9.0              |24_linux64_openblas          15 KB  conda-forge
    libcblas-3.9.0             |24_linux64_openblas          15 KB  conda-forge
    libgfortran-14.1.0         |       h69a702a_1          51 KB  conda-forge
    libgfortran-ng-14.1.0      |       h69a702a_1          51 KB  conda-forge
    libgfortran5-14.1.0        |       hc5f4f2c_1         1.4 MB  conda-forge
    liblapack-3.9.0            |24_linux64_openblas          15 KB  conda-forge
    libopenblas-0.3.27         |pthreads_hac2b453_1         5.3 MB  conda-forge
    numpy-2.1.2                |  py312h58c1407_0         8.1 MB  conda-forge
    ------------------------------------------------------------
                                         

In [42]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

PyCall.PyError: PyError (PyImport_ImportModule

The Python package numpy could not be imported by pyimport. Usually this means
that you did not install numpy in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/usr/bin/python3

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the numpy module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.build("PyCall"), and re-lau

In [43]:
numpy_sum(a)

UndefVarError: UndefVarError: `numpy_sum` not defined

In [44]:
numpy_sum(a) ≈ sum(a)

UndefVarError: UndefVarError: `numpy_sum` not defined

In [45]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

UndefVarError: UndefVarError: `py_numpy_bench` not defined

# 5. Python, hand-written 

In [31]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

LoadError: LoadError: UndefVarError: `@py_str` not defined
in expression starting at /hm2/code/julia/JuliaAcademy/Julia_Intro/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X65sZmlsZQ==.jl:1

In [36]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.012 s …  1.027 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.018 s             ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.019 s ± 6.023 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █       █              █        █                      █  
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  110 s         Histogram: frequency by time         130 s <

 Memory estimate: 336 bytes, allocs estimate: 6.

In [37]:
sum_py(a)

5.000406093458116e6

In [38]:
sum_py(a) ≈ sum(a)

true

In [39]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "C"                   => 9.38288
  "Python numpy"        => 1.91263
  "Python hand-written" => 1011.92
  "Python built-in"     => 875.812
  "C -ffast-math"       => 1.78121

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [32]:
@which sum(a)

sum(a::AbstractArray; dims, kw...)
     @ Base reducedim.jl:1010

In [33]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 1236 samples with 1 evaluation.
 Range (min … max):  2.629 ms … 9.128 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.695 ms             ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.030 ms ± 1.012 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▃▄██▆█▆▄▄▂                                            
  ▂▃▄▆█████████████▇▇▆▅▆▆▃▅▅▄▅▅▅▄▄▃▃▃▃▃▃▄▄▃▃▃▃▂▂▂▂▂▂▂▂▃▁▂▃▂ ▄
  2.63 ms        Histogram: frequency by time       7.53 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [34]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "C"              => 6.95362
  "Julia built-in" => 2.62913

# 7. Julia (hand-written) 

In [35]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [36]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 650 samples with 1 evaluation.
 Range (min … max):  6.861 ms …  11.866 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.440 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.676 ms ± 688.880 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▃▇█▄▇▆▇▃▅ ▂                                                
  ▄▇███████████▇▆▆▇▇█▇▅▃▇▅▆▄▅▃▆▃▄▄▅▄▄▄▅▄▃▂▃▁▄▃▃▂▂▃▁▂▂▁▁▂▁▂▃▁▂ ▄
  6.86 ms         Histogram: frequency by time          10 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [45]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 7 entries:
  "C"                   => 9.38288
  "Python numpy"        => 1.91263
  "Julia hand-written"  => 9.387
  "Python hand-written" => 1011.92
  "Python built-in"     => 875.812
  "Julia built-in"      => 1.8605
  "C -ffast-math"       => 1.78121

# 8. Julia (hand-written w. simd) 

In [46]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [47]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 2023 samples with 1 evaluation.
 Range (min … max):  2.370 ms …  3.173 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.429 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.454 ms ± 67.874 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

        █▇▆▅▄▄▃▃▃▃▂▂▂▂    ▁ ▁▁                               ▁
  █▄▆██████████████████████▇██▇▆▇▆▇▆▆▆▆▆▄▆▅▆▅▆▄▄▁▄▄▄▄▄▄▅▄▄▄▄ █
  2.37 ms      Histogram: log(frequency) by time     2.76 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [48]:
mysum_simd(a)

5.000406093458086e6

In [49]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any, Any} with 8 entries:
  "Julia hand-written simd" => 2.37012
  "C"                       => 9.38288
  "Python numpy"            => 1.91263
  "Julia hand-written"      => 9.387
  "Python hand-written"     => 1011.92
  "Python built-in"         => 875.812
  "Julia built-in"          => 1.8605
  "C -ffast-math"           => 1.78121

# Summary

In [50]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

C -ffast-math...............1.8
Julia built-in..............1.9
Python numpy................1.9
Julia hand-written simd.....2.4
C...........................9.4
Julia hand-written..........9.4
Python built-in...........875.8
Python hand-written......1011.9
